In [72]:
import requests
import lxml
import pandas as pd
from bs4 import BeautifulSoup # Html search tool
import re # Regular expressions

In [ ]:
response = requests.get("https://www.zillow.com/homes/Raleigh,-NC_rb/")
print(response.text)

In [125]:
# Header required to circumvent captcha
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as main_session:
   city_state_abbr = 'raleigh-nc/' #*****change this city to what you want*****
   url = 'https://www.zillow.com/' + city_state_abbr
   main_request = main_session.get(url, headers=req_headers)

In [136]:
#add contents of urls to soup variable from each url
soup = BeautifulSoup(main_request.content, "html.parser")

In [ ]:
# After finding urls from the main page, use those links to open homedetaill page
with requests.Session() as branch_session:
    branch_base_url = "https://www.zillow.com/homedetails/"
    branch_sub_url = "5904-Endsley-Ct-Raleigh-NC-27610/65332868_zpid/"

    branch_request = branch_session.get(branch_base_url + branch_sub_url, headers=req_headers)

branch_soup = BeautifulSoup(branch_request.content, "html.parser")


branch_soup.prettify()

In [185]:
raw_price_soup = branch_soup.find_all("span", class_ = "Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY")
raw_bed_soup = branch_soup.find_all("strong")
raw_address_soup = branch_soup.find("h1")

print(raw_price_soup) # done
print(raw_bed_soup) # done
print(raw_address_soup)

[<span class="Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY" data-testid="price"><span>$352,000</span></span>]
[<strong>3</strong>, <strong>3</strong>, <strong>1,566</strong>, <strong>6 days</strong>, <strong>3,532</strong>, <strong>200</strong>, <strong>MON</strong>, <strong>TUE</strong>, <strong>WED</strong>, <strong>THU</strong>]
<h1 class="Text-c11n-8-73-0__sc-aiai24-0 kHeRng">5904 Endsley Ct,<!-- --> <!-- -->Raleigh, NC 27610</h1>


In [186]:
# Price
raw_price_soup = str(branch_soup.find_all("span", class_ = "Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY"))
price_match_object = re.search("<span>(.+?)</span>", raw_price_soup)
price = price_match_object.group(1)

print(raw_price_soup)
print(price)

[<span class="Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY" data-testid="price"><span>$352,000</span></span>]
$352,000


In [184]:
# Get all attributes besides price and address
# Turn this into a function bb

# 0 Bed, 1 bath,  2 sqft, 3 days on zillow, 5 views, 6 saves
raw_bed_soup = str(branch_soup.find_all("strong"))
raw_bed_soup_list = raw_bed_soup.split(' ')
count_beds = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[0]).group(1)
count_baths = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[1]).group(1)
sq_ft = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[2]).group(1)
days_on_zillow = re.search("<strong>(.+?)", raw_bed_soup_list[3]).group(1)
count_views = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[5]).group(1)
count_saves = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[6]).group(1)


print("beds",count_beds)
print("baths", count_baths)
print("sq_ft", sq_ft)
print("days_on_zillow", days_on_zillow)
print("count_views", count_views)
print("count_saves", count_saves)

beds 3
baths 3
sq_ft 1,566
days_on_zillow 6
count_views 3,532
count_saves 200


In [195]:
print(raw_address_soup)

<h1 class="Text-c11n-8-73-0__sc-aiai24-0 kHeRng">5904 Endsley Ct,<!-- --> <!-- -->Raleigh, NC 27610</h1>


In [215]:
address = str(raw_address_soup)
address_1 = re.search("kHeRng(.+?)<!--", address).group(1)
address_2 = re.search("<!-- -->(.+?)</h1>", address).group(1)


print(address_1)
print(address_2)

">5904 Endsley Ct,
 <!-- -->Raleigh, NC 27610


In [ ]:
# From intial queried page (e.g., homes in Raleigh) find all links to homedetails page,
# which will have more detailed information on the home.

# This will be the main base where the scraper will come back to after scraping each
# home details page. 

# Example: queried page -> homedetails page -> scrape data -> queried page 
# Loop until all homes are scraped, then proceed to next page.
soup.find_all(href=re.compile("homedetails"))
# soup.select('a[href^="https://www.zillow.com/homedetails/"]')


In [129]:
def find_all_keyword_units(keyword):
    """Returns a list of keyword matches"""
    keyword_match_list = soup.find_all(href=re.compile(str(keyword)))
    return keyword_match_list


# Create function to take list and return pruned list including only urls for homedetails
def strip_list_to_url(html_list, index=5):
    list_to_str = str(html_list)
    href_url = list_to_str.split(' ')[index] # Selects index of href with housedetails
    href_url_strip = href_url.replace("href=","")
    href_url_strip = href_url_strip.replace('"','')
    return href_url_strip

print(strip_list_to_url(find_all_keyword_units("homedetails")))

"""
for count, value in enumerate(test_output_str.split(' ')):
    print(count, value)"""


https://www.zillow.com/homedetails/2717-Hiking-Trl-Raleigh-NC-27615/6495378_zpid/


"\nfor count, value in enumerate(test_output_str.split(' ')):\n    print(count, value)"

In [ ]:
# Simply testing function, can remove
test_output = find_all_keyword_units(keyword = "homedetails")
print(test_output)

https://www.zillow.com/homedetails/5904-Endsley-Ct-Raleigh-NC-27610/65332868_zpid/


"\nfor count, value in enumerate(test_output_str.split(' ')):\n    print(count, value)"

only difference is 2_p for page 2 3_p for page 3

https://www.zillow.com/raleigh-nc/luxury-homes/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D

https://www.zillow.com/raleigh-nc/luxury-homes/2_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D

https://www.zillow.com/raleigh-nc/luxury-homes/3_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22pagination%22%3A%7B%22currentPage%22%3A3%7D%7D

In [23]:
# https://blog.devgenius.io/scraping-zillow-with-python-and-beautifulsoup-bbc7e581c218

#create the first two dataframes
df = pd.DataFrame()
df1 = pd.DataFrame()
#all for loops are pulling the specified variable using beautiful soup and inserting into said variable
for i in soup:
    address = soup.find_all (class_= 'list-card-addr')
    price = list(soup.find_all (class_='list-card-price'))
    beds = list(soup.find_all("ul", class_="list-card-details"))
    details = soup.find_all ('div', {'class': 'list-card-details'})
    home_type = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link = soup.find_all (class_= 'list-card-link')
    
    #create dataframe columns out of variables
    df['prices'] = price
    df['address'] = address
    df['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df['links'] = urls
df['links'] = df['links'].astype('str')

#remove html tags
df['links'] = df['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df['links'] = df['links'].replace('" tabindex="0"></a>', ' ', regex=True)

AttributeError: 'NoneType' object has no attribute 'find'